In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16099552019600988268,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1392391390617979367
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1914467379075914557
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:XLA_GPU:1"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6850332535468460366
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:XLA_GPU:2"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15647959244670680162
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:XLA_GPU:3"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16942679776521885520
 physical_device_desc: "device: XLA_GPU device",

In [28]:
import sys
sys.path.insert(1, '/p/project/joaiml/CovidNet')

In [29]:
import cv2
import time
import pandas as pd
import numpy as np
import tensorflow
from data import process_image_file
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [30]:
from IPython.display import display, clear_output

In [31]:
def inference(imagepath):

    weightspath = '/p/project/joaiml/CovidNet/models/COVIDNet-CXR4-A'
    metaname = 'model.meta'
    ckptname = 'model-18540'
    in_tensorname = 'input_1:0'
    out_tensorname = 'norm_dense_1/Softmax:0'
    input_size = 480
    top_percent = 0.08

    mapping = {'normal': 0, 'pneumonia': 1, 'COVID-19': 2}
    inv_mapping = {0: 'normal', 1: 'pneumonia', 2: 'COVID-19'}

    tensorflow.reset_default_graph()
    sess = tensorflow.Session()
    tensorflow.get_default_graph()
    saver = tensorflow.train.import_meta_graph(os.path.join(weightspath, metaname))
    saver.restore(sess, os.path.join(weightspath, ckptname))

    graph = tensorflow.get_default_graph()

    image_tensor = graph.get_tensor_by_name(in_tensorname)
    pred_tensor = graph.get_tensor_by_name(out_tensorname)

    x = process_image_file(imagepath, top_percent, input_size)
    x = x.astype('float32') / 255.0
    #
    pred = sess.run(pred_tensor, feed_dict={image_tensor: np.expand_dims(x, axis=0)})
    # display the diff
    
    prediction = inv_mapping[pred.argmax(axis=1)[0]]

    return prediction, (format(pred[0][0], '.3f'), format(pred[0][1], '.3f'), format(pred[0][2], '.3f'))

In [32]:
DATAFRAME_NAME = '/p/project/joaiml/CovidNet/inference_covidx_dataset_results_deep_jupyter.csv'
df = pd.read_csv(DATAFRAME_NAME)

In [33]:
df.head()

,Image,Correct,Predicted,Confidence_Normal,Confidence_Pneumonia,Confidence_Covid19


In [34]:
img_paths = ['/p/project/joaiml/CovidNet/COVIDx/test/COVID-19',
             '/p/project/joaiml/CovidNet/COVIDx/test/normal',
             '/p/project/joaiml/CovidNet/COVIDx/test/pneumonia']

In [ ]:
total = time.time()
for path in img_paths:
    images = os.listdir(path)
    for idx, image in enumerate(images):
        path_to_image = os.path.join(path, image)
        if path_to_image not in df['Image'].values:
            clear_output(wait=True)
            display('Inference on image {} / {}. Time elapsed this run: {}'.format(idx+1, len(images), format(time.time()-total)))
            res, conf = inference(path_to_image)
            df = df.append({'Image': path_to_image,
                            'Correct': path_to_image.split('/')[-2],
                            'Predicted': res,
                            'Confidence_Normal': conf[0],
                            'Confidence_Pneumonia': conf[1],
                            'Confidence_Covid19': conf[2]}, ignore_index=True)
            df.to_csv(DATAFRAME_NAME, index=False)

display("Finished inference on CovidX. Time elapsed: {}".format(time.time()-total))

'Inference on image 45 / 100. Time elapsed this run: 966.7587938308716'

INFO:tensorflow:Restoring parameters from /p/project/joaiml/CovidNet/models/COVIDNet-CXR4-A/model-18540


In [25]:
DATAFRAME_NAME = '/p/project/joaiml/CovidNet/inference_ehl_dataset_results_deep_jupyter.csv'
df = pd.read_csv(DATAFRAME_NAME)

In [27]:
df.head()

,Image,Correct,Predicted,Confidence_Normal,Confidence_Pneumonia,Confidence_Covid19


In [26]:
img_paths = ['/p/project/joaiml/CovidNet/EHL/test/COVID-19',
             '/p/project/joaiml/CovidNet/EHL/train/COVID-19',
             '/p/project/joaiml/CovidNet/EHL/test/Normal',
             '/p/project/joaiml/CovidNet/EHL/train/Normal',
             '/p/project/joaiml/CovidNet/EHL/test/Pneumonia',
             '/p/project/joaiml/CovidNet/EHL/train/Pneumonia']

In [ ]:
total = time.time()
for path in img_paths:
    images = os.listdir(path)
    for idx, image in enumerate(images):
        path_to_image = os.path.join(path, image)
        if path_to_image not in df['Image'].values:
            clear_output(wait=True)
            display('Inference on image ({}) {} / {}. Time elapsed this run: {}'.format(path_to_image, idx+1, len(images), format(time.time()-total)))
            res, conf = inference(path_to_image)
            df = df.append({'Image': path_to_image,
                            'Correct': path_to_image.split('/')[-2],
                            'Predicted': res,
                            'Confidence_Normal': conf[0],
                            'Confidence_Pneumonia': conf[1],
                            'Confidence_Covid19': conf[2]}, ignore_index=True)
            df.to_csv(DATAFRAME_NAME, index=False)

In [20]:
"""
df = pd.DataFrame(columns = ['Image', 'Correct', 'Predicted', 'Confidence_Normal', 'Confidence_Pneumonia', 'Confidence_Covid19'])
df.to_csv('/p/project/joaiml/CovidNet/inference_covidx_dataset_results_deep_jupyter.csv', index=False)
"""

"\ndf = pd.DataFrame(columns = ['Image', 'Correct', 'Predicted', 'Confidence_Normal', 'Confidence_Pneumonia', 'Confidence_Covid19'])\ndf.to_csv('/p/project/joaiml/CovidNet/inference_covidx_dataset_results_deep_jupyter.csv', index=False)\n"